In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np

In [2]:
#Import libraries and dependencies

import numpy as np
import pandas as pd
from pathlib import Path

Read CSV into pandas dataframe

In [3]:
# Load env data
load_dotenv()


True

In [4]:
# Assign Alpaca keys to variables
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


In [5]:
# Initialize Alpaca connection
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)


In [6]:
# Set start and end dates for
start_date = pd.Timestamp('2020-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-12-31', tz='America/New_York').isoformat()


In [7]:
# Set file path:

#filepath = Path("../Resources/JPM.csv")

In [8]:
# Assign default ticker symbol
ticker_string = 'JPM'
ticker = [ticker_string]


In [9]:
# Assign default time frame
timeframe = '15Min'

In [10]:
# Get ticker historical data
df_ohlc_original = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df


In [11]:
# Copy of original data
df_ohlc = df_ohlc_original

# Verify data in dataframe
df_ohlc = df_ohlc[ticker_string]
df_ohlc.head()


,open,high,low,close,volume
time,,,,,
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695


In [12]:
#jpm_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

#jpm_df.head()

In [13]:
#jpm_df.rename(columns={"Unnamed: 0": 'date'}, inplace=True)

In [14]:
#jpm_df.set_index("date",inplace =True)

In [15]:
#jpm_df.head()

In [16]:
"""
p_length = 20

def truehigh(h,yh):
    if yh >= h :
        truehigh_val= yh
    else:
        truehigh_val = h
    
    return truehigh_val



#df_ohlc['trueHigh'] = np.where(((df_ohlc['high'] > df_ohlc['truehigh'].shift(1))),df_ohlc['high'],df_ohlc['truehigh'])
"""


In [18]:
df_ohlc.head()

,open,high,low,close,volume
time,,,,,
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695


In [30]:
from stockstats import StockDataFrame as sdf

stockstats_df = sdf.retype(df_ohlc)

In [21]:
class StockDataFrame(pd.DataFrame):
    OPERATORS = ['le', 'ge', 'lt', 'gt', 'eq', 'ne']

    # Start of options.
    KDJ_PARAM = (2.0 / 3.0, 1.0 / 3.0)
    KDJ_WINDOW = 9

    BOLL_PERIOD = 20
    BOLL_STD_TIMES = 2

    MACD_EMA_SHORT = 12
    MACD_EMA_LONG = 26
    MACD_EMA_SIGNAL = 9

In [25]:
@classmethod
def _get_macd(cls, df):
    """ Moving Average Convergence Divergence
    This function will initialize all following columns.
    MACD Line (macd): (12-day EMA - 26-day EMA)
    Signal Line (macds): 9-day EMA of MACD Line
    MACD Histogram (macdh): MACD Line - Signal Line
    :param df: data
    :return: None
    """
    ema_short = 'close_{}_ema'.format(cls.MACD_EMA_SHORT)
    ema_long = 'close_{}_ema'.format(cls.MACD_EMA_LONG)
    ema_signal = 'macd_{}_ema'.format(cls.MACD_EMA_SIGNAL)
    fast = df[ema_short]
    slow = df[ema_long]
    df['macd'] = fast - slow
    df['macds'] = df[ema_signal]
    df['macdh'] = (df['macd'] - df['macds'])
    log.critical("NOTE: Behavior of MACDH calculation has changed as of "
                     "July 2017 - it is now 1/2 of previous calculated values")
    cls._drop_columns(df, [ema_short, ema_long, ema_signal])

In [27]:
_get_macd(df_ohlc)

TypeError: 'classmethod' object is not callable

In [31]:


stock = sdf.retype(df_ohlc)
print(stock['macds'])

print(stock)

time
2020-01-02 09:30:00-05:00    0.000000
2020-01-02 09:45:00-05:00    0.000000
2020-01-02 10:00:00-05:00   -0.000381
2020-01-02 10:15:00-05:00   -0.003638
2020-01-02 10:30:00-05:00   -0.005334
                               ...   
2020-12-30 14:45:00-05:00    0.000626
2020-12-30 15:00:00-05:00   -0.006236
2020-12-30 15:15:00-05:00   -0.011130
2020-12-30 15:30:00-05:00   -0.011477
2020-12-30 15:45:00-05:00   -0.012439
Name: macds, Length: 6773, dtype: float64
                              open    high     low    close  volume      macd  \
time                                                                            
2020-01-02 09:30:00-05:00  140.070  140.14  139.52  139.670   29953  0.000000   
2020-01-02 09:45:00-05:00  139.675  139.91  139.44  139.670   24343  0.000000   
2020-01-02 10:00:00-05:00  139.635  139.81  139.52  139.640   19021 -0.000930   
2020-01-02 10:15:00-05:00  139.600  139.60  139.34  139.400   18856 -0.009996   
2020-01-02 10:30:00-05:00  139.460  139.54  139.2

In [32]:
print(stock['atr'])
print(stock)

time
2020-01-02 09:30:00-05:00         NaN
2020-01-02 09:45:00-05:00    0.470000
2020-01-02 10:00:00-05:00    0.376667
2020-01-02 10:15:00-05:00    0.349196
2020-01-02 10:30:00-05:00    0.324360
                               ...   
2020-12-30 14:45:00-05:00    0.219264
2020-12-30 15:00:00-05:00    0.211460
2020-12-30 15:15:00-05:00    0.203498
2020-12-30 15:30:00-05:00    0.211106
2020-12-30 15:45:00-05:00    0.228884
Name: atr, Length: 6773, dtype: float64
                              open    high     low    close  volume      macd  \
time                                                                            
2020-01-02 09:30:00-05:00  140.070  140.14  139.52  139.670   29953  0.000000   
2020-01-02 09:45:00-05:00  139.675  139.91  139.44  139.670   24343  0.000000   
2020-01-02 10:00:00-05:00  139.635  139.81  139.52  139.640   19021 -0.000930   
2020-01-02 10:15:00-05:00  139.600  139.60  139.34  139.400   18856 -0.009996   
2020-01-02 10:30:00-05:00  139.460  139.54  139.28 

In [33]:
print(stock['atr'])

time
2020-01-02 09:30:00-05:00         NaN
2020-01-02 09:45:00-05:00    0.470000
2020-01-02 10:00:00-05:00    0.376667
2020-01-02 10:15:00-05:00    0.349196
2020-01-02 10:30:00-05:00    0.324360
                               ...   
2020-12-30 14:45:00-05:00    0.219264
2020-12-30 15:00:00-05:00    0.211460
2020-12-30 15:15:00-05:00    0.203498
2020-12-30 15:30:00-05:00    0.211106
2020-12-30 15:45:00-05:00    0.228884
Name: atr, Length: 6773, dtype: float64


In [34]:
stock['atr']

time
2020-01-02 09:30:00-05:00         NaN
2020-01-02 09:45:00-05:00    0.470000
2020-01-02 10:00:00-05:00    0.376667
2020-01-02 10:15:00-05:00    0.349196
2020-01-02 10:30:00-05:00    0.324360
                               ...   
2020-12-30 14:45:00-05:00    0.219264
2020-12-30 15:00:00-05:00    0.211460
2020-12-30 15:15:00-05:00    0.203498
2020-12-30 15:30:00-05:00    0.211106
2020-12-30 15:45:00-05:00    0.228884
Name: atr, Length: 6773, dtype: float64

In [36]:
atr_column = pd.DataFrame(stock['atr'])

In [37]:
atr_column.head()

,atr
time,
2020-01-02 09:30:00-05:00,NaN
2020-01-02 09:45:00-05:00,0.470000
2020-01-02 10:00:00-05:00,0.376667
2020-01-02 10:15:00-05:00,0.349196
2020-01-02 10:30:00-05:00,0.324360


In [38]:
df_final = df_ohlc.join(atr_column)

In [39]:
df_final.head()

,open,high,low,close,volume,close_12_ema,close_26_ema,macd,macd_9_ema,macds,macdh,atr
time,,,,,,,,,,,,
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953,139.670000,139.670000,0.000000,0.000000,0.000000,0.000000,NaN
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343,139.670000,139.670000,0.000000,0.000000,0.000000,0.000000,0.470000
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021,139.658291,139.659221,-0.000930,-0.000381,-0.000381,-0.000549,0.376667
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856,139.576759,139.586754,-0.009996,-0.003638,-0.003638,-0.006358,0.349196
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695,139.565413,139.574752,-0.009339,-0.005334,-0.005334,-0.004005,0.324360


In [45]:
p_length = 20
df_final['kcmid'] = df_final['close'].rolling(window=p_length).mean()
df_final['kcup'] = df_final['kcmid']+df_final['atr']
df_final['kclo'] = df_final['kcmid']-df_final['atr']
df_final.tail()

,open,high,low,close,volume,close_12_ema,close_26_ema,macd,macd_9_ema,macds,macdh,atr,kcmid,kcup,kclo
time,,,,,,,,,,,,,,,
2020-12-30 14:45:00-05:00,125.285,125.46,125.26,125.29,7778,125.361912,125.393819,-0.031907,0.000626,0.000626,-0.032533,0.219264,125.4420,125.661264,125.222736
2020-12-30 15:00:00-05:00,125.325,125.36,125.25,125.31,5098,125.353925,125.387610,-0.033685,-0.006236,-0.006236,-0.027448,0.211460,125.4565,125.667960,125.245040
2020-12-30 15:15:00-05:00,125.350,125.41,125.32,125.36,3735,125.354860,125.385565,-0.030705,-0.011130,-0.011130,-0.019575,0.203498,125.4590,125.662498,125.255502
2020-12-30 15:30:00-05:00,125.410,125.67,125.37,125.55,7877,125.384881,125.397745,-0.012864,-0.011477,-0.011477,-0.001387,0.211106,125.4520,125.663106,125.240894
2020-12-30 15:45:00-05:00,125.520,125.71,125.25,125.33,34033,125.376438,125.392727,-0.016289,-0.012439,-0.012439,-0.003850,0.228884,125.4390,125.667884,125.210116


In [46]:
drop_columns = ['close_12_ema','close_26_ema','macd', 'macd_9_ema', 'macds', 'macdh','kcmid']
df_final.drop(drop_columns, inplace=True, axis=1)
df_final.tail()

,open,high,low,close,volume,atr,kcup,kclo
time,,,,,,,,
2020-12-30 14:45:00-05:00,125.285,125.46,125.26,125.29,7778,0.219264,125.661264,125.222736
2020-12-30 15:00:00-05:00,125.325,125.36,125.25,125.31,5098,0.211460,125.667960,125.245040
2020-12-30 15:15:00-05:00,125.350,125.41,125.32,125.36,3735,0.203498,125.662498,125.255502
2020-12-30 15:30:00-05:00,125.410,125.67,125.37,125.55,7877,0.211106,125.663106,125.240894
2020-12-30 15:45:00-05:00,125.520,125.71,125.25,125.33,34033,0.228884,125.667884,125.210116
